# 2. Evolve only, time dependent.

## To do:
* change into time dependent code
 * test for time independent solution
 * then test for time dependent solution

In [2]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import math as m
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])
# Exact Solution
c = 16.0 #*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

ue = sym.simplify(iliescu)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = 0 #sym.diff(ue,t) Remove time dependence for now
fe += sym.diff(ue,t) - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

# print('u_code = ' + u_code + '\n')
# print('f_code = ' + f_code)

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	errors = {'L2 norm': L2n, 'H1 norm': H1n}
	return 'L2, ' + str(L2n) +', H1, '+ str(H1n) +', t, '+ str(t) +'\n'

In [3]:
def evolve_t(nx, dt, T, u_code, f_code, sigma, mu, velocity):
    degree = 2
    
    t = 0.0
    u_exact = Expression(u_code, degree = degree, t=t)
    f = Expression(f_code, degree = degree, t = dt)

    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree, t=0.0)
    def boundary(x, on_boundary):
        return on_boundary
    
    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = (u - u_n)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    while t - T < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        b = assemble(L)
        bc.apply(b)
        solve(A1, u_.vector(), b, 'gmres')
        progress.update(t / T)
        
        # Update current time
        t += dt
        f.t += dt
        u_D.t += dt
    
    errors = compute_errors(u_exact, u_, 0, mesh)
    print(errors)
    
    out_file_uexact = File("time_dep/evolve_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << (ue, float(t))
    out_file_ubar = File("time_dep/"+str(mu)+"evolve_u_"+str(nx)+".pvd") 
    out_file_ubar << (u_, float(t))

In [9]:
dt=0.01
T=0.02
for nx in [25, 50, 100]:
    print('nx = '+str(nx))
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

nx = 25
L2, 0.368993791938, H1, 10.6678477777, t, 0

nx = 50
L2, 0.365904801733, H1, 11.3773983927, t, 0

nx = 100
L2, 0.364453616144, H1, 7.53380792097, t, 0



In [10]:
for nx in [200, 400]:
    print('nx = '+str(nx))
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

nx = 200
L2, 0.364351562791, H1, 4.60927008799, t, 0

nx = 400
L2, 0.364348676342, H1, 4.25925064503, t, 0



In [ ]:
nx = 800
print('nx = '+str(nx))
evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

nx = 800


## Exact Solution
* c = 16.0 #*sym.sin(sym.pi*t)
* h = x*(1-x)*y*(1-y)
* g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
 * iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

nx = [25, 50, 100, 200, 400, 800];


## Problem Parameters
* sigma = 1.0
* b=as_vector([2.0, 3.0])
* mu = 10**(-5)

### Case: no time dependence solution (solution from step 1)
* dt = 0.01, T = 0.02: checking the first time step solution converges